In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev7
cdv, version 1.1.2
Python 3.10.6


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])

In [3]:
# from setup-sim notebook
# alice 4070016934
# bob 861183802
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])

In [4]:
%%bash
cdv rpc coinrecords --by puzzlehash "4630b5ab9143ca8766fe8c588cdd09f4baf79ffea0e882ff11b55c9e03086931"
cdv rpc coinrecords --by puzzlehash "f7f6d4109f8357c89319815ffc4cc130640a249dcd63575481a22a4d96b660f7"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x4630b5ab9143ca8766fe8c588cdd09f4baf79ffea0e882ff11b55c9e03086931"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665127011
    }
]
[
    {
        "coin": {
            "amount": 1000000000000,
            "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
            "puzzle_hash": "0xf7f6d4109f8357c89319815ffc4cc130640a249dcd63575481a22a4d96b660f7"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665127011
    }
]


# Create Game Coin
> Two waiting room coins available from Alice and Bob

[waiting-room.clsp](../code/waiting-room.clsp)
```clojure
(mod (OWNER_PUZZLE_HASH PK action)
```

In [5]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    [alice_puzzle_hash, alice_pk],
    [alice_pk, bob_pk]
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch1gcctt2u3g09gweh733vgehgf7ja008l75r5g9lc3k4wfuqcgdycs05ez62'

In [6]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    [bob_puzzle_hash, bob_pk],
    [alice_pk, bob_pk]
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
bob_waiting_room_address = encode_puzzle_hash(bob_waiting_room_puzzle_hash, "txch")
bob_waiting_room_address

'txch17lmdgyylsdtu3yces90lcnxpxpjq5fyae434w4yp5g4ym94kvrmsj84h2w'

## Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)
```clojure
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
```

In [7]:
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = (player_amount * 2) + 1 # odd amount for singleton
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

## Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)
```clojure
(mod (MOD TERMINATE_PUZZLE PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
```

In [8]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## Launcher Coin

In [9]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = "mainnet"
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_coin = coin_records[0].coin
print(f'alice_waiting_room_coin:\n {alice_waiting_room_coin.to_json_dict()}')
print()
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(bob_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

bob_waiting_room_coin = coin_records[0].coin
print(f'bob_waiting_room_coin:\n {bob_waiting_room_coin.to_json_dict()}')

alice_waiting_room_coin:
 {'parent_coin_info': '0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b', 'puzzle_hash': '0x4630b5ab9143ca8766fe8c588cdd09f4baf79ffea0e882ff11b55c9e03086931', 'amount': 1000000000001}

bob_waiting_room_coin:
 {'parent_coin_info': '0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97', 'puzzle_hash': '0xf7f6d4109f8357c89319815ffc4cc130640a249dcd63575481a22a4d96b660f7', 'amount': 1000000000000}


In [10]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id.hex()}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
launcher_coin


launcher id: db13d76b91657651356b19e83c7573fa54fb8c6572c6d24b0ee8f5e2b62077fb


Coin { parent_coin_info: 18bc6ee8655344ffb0de25683cfee16b661aa28cfea66ae508327da877ec4a8e, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }

### Singleton Launcher
[singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm)
[singleton_launcher.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_launcher.clvm)
```clojure
(mod (singleton_full_puzzle_hash amount key_value_list)
```

In [11]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic-tac-toe"), 
            ("p1_pk", alice_pk),
            ("p2_pk", bob_pk)
        ]
    ]
)
print_program(launcher_solution)

launcher_announcement = launcher_solution.get_tree_hash()
print(f'\nlauncher_announcement:\n{launcher_announcement}')

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

(0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))

launcher_announcement:
43bc64fafddcee2b9788769a986277659d93dded0e908c18d9408fef201f17a8
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0x18bc6ee8655344ffb0de25683cfee16b661aa28cfea66ae508327da877ec4a8e",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff80808080

In [12]:
%%bash
opd ff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080
opd ffa0c8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35bafff8601d1a94a2001ffffff8467616d658b7469632d7461632d746f65ffff8570315f706bb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff8570325f706bb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a8080

(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))
(0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))


In [13]:
%%bash
brun '(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' '(0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))'

((51 0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001) (60 0x43bc64fafddcee2b9788769a986277659d93dded0e908c18d9408fef201f17a8))


In [14]:
print(f'launcher_announcement:\n{launcher_announcement}')
print(f'\nlauncher_coin_id:\n{launcher_id.hex()}')

# announcementID == sha256(coinID + message).
from chia.util.hash import std_hash
launcher_coin_announcement = std_hash(launcher_id + launcher_announcement)
print(f'\nlauncher_coin_announcement:\n{launcher_coin_announcement}')

launcher_announcement:
43bc64fafddcee2b9788769a986277659d93dded0e908c18d9408fef201f17a8

launcher_coin_id:
db13d76b91657651356b19e83c7573fa54fb8c6572c6d24b0ee8f5e2b62077fb

launcher_coin_announcement:
d68fe586e06f717e167d98586a7f0d58fbdbf2bc20b8699ec7837afcfdefa1a4


In [15]:
alice_waiting_room_coin_spend = CoinSpend(
    alice_waiting_room_coin,
    alice_waiting_room_puzzle,
    Program.to([
        [
            singleton_puzzle.get_tree_hash(),
            game_amount
        ],
        1 # CREATE-GAME
    ])
)
print_json(alice_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
        "puzzle_hash": "0x4630b5ab9143ca8766fe8c588cdd09f4baf79ffea0e882ff11b55c9e03086931"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff3cff2f80ffff01ff04ffff04ff2cffff01ff648080ffff04ffff04ff12ffff04ff09ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff15ffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ffff02ff2effff04ff02ffff04ff27ffff04ff57ffff04ff13ffff04ff2bff80808080808080ff808080ffff02ffff03ffff22ffff09ff2aff2f80ffff09ff15ff138080ffff01ff04ffff04ff12ffff04ff16ffff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3dff5281ffffff33ff01ff8467616d658b7469632d7461632d746f65ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff3effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ff3affff04ffff04ffff018570315f706bff1780ffff04ffff04ff

In [16]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff09ff3cff2f80ffff01ff04ffff04ff2cffff01ff648080ffff04ffff04ff12ffff04ff09ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff15ffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ffff02ff2effff04ff02ffff04ff27ffff04ff57ffff04ff13ffff04ff2bff80808080808080ff808080ffff02ffff03ffff22ffff09ff2aff2f80ffff09ff15ff138080ffff01ff04ffff04ff12ffff04ff16ffff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3dff5281ffffff33ff01ff8467616d658b7469632d7461632d746f65ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff3effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ff3affff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff3effff04ff02ffff04ff09ff80808080ffff02ff3effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01ffa0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb101080ffff04ffff01ffb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a80ff01808080
opd ffffa0c8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35bafff8601d1a94a200180ff0180

(a (q 2 (q 2 (i (= 60 47) (q 4 (c 44 (q 100)) (c (c 18 (c 9 (q 0x00e8d4a51000))) (c (c 16 (c 21 (c 24 ()))) ()))) (q 4 (c 20 (c (a 46 (c 2 (c 39 (c 87 (c 19 (c 43 ())))))) ())) (a (i (all (= 42 47) (= 21 19)) (q 4 (c 18 (c 22 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 82 . -1) (51 1 "game" . "tic-tac-toe") 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 (a 62 (c 2 (c (c 5 (c 11 (c (c 58 (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) 2 (i (l 5) (q 11 (q . 2) (a 62 (c 2 (c 9 ()))) (a 62 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))
((0xc8e26acf4260

In [17]:
%%bash
brun '(a (q 2 (q 2 (i (= 60 47) (q 4 (c 44 (q 100)) (c (c 18 (c 9 (q 0x00e8d4a51000))) (c (c 16 (c 21 (c 24 ()))) ()))) (q 4 (c 20 (c (a 46 (c 2 (c 39 (c 87 (c 19 (c 43 ())))))) ())) (a (i (all (= 42 47) (= 21 19)) (q 4 (c 18 (c 22 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 82 . -1) (51 1 "game" . "tic-tac-toe") 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 (a 62 (c 2 (c (c 5 (c 11 (c (c 58 (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) 2 (i (l 5) (q 11 (q . 2) (a 62 (c 2 (c 9 ()))) (a 62 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))' '((0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001) 1)'

((61 0x43bc64fafddcee2b9788769a986277659d93dded0e908c18d9408fef201f17a8) (51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001))


In [18]:
bob_waiting_room_coin_spend = CoinSpend(
    bob_waiting_room_coin,
    bob_waiting_room_puzzle,
    Program.to([
        [
            singleton_puzzle.get_tree_hash(),
            game_amount
        ],
        0 # SPEND
    ])
)
print_json(bob_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000000,
        "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
        "puzzle_hash": "0xf7f6d4109f8357c89319815ffc4cc130640a249dcd63575481a22a4d96b660f7"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff09ff3cff2f80ffff01ff04ffff04ff2cffff01ff648080ffff04ffff04ff12ffff04ff09ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff15ffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ffff02ff2effff04ff02ffff04ff27ffff04ff57ffff04ff13ffff04ff2bff80808080808080ff808080ffff02ffff03ffff22ffff09ff2aff2f80ffff09ff15ff138080ffff01ff04ffff04ff12ffff04ff16ffff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3dff5281ffffff33ff01ff8467616d658b7469632d7461632d746f65ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff3effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ff3affff04ffff04ffff018570315f706bff1780ffff04ffff04ff

In [19]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff09ff3cff2f80ffff01ff04ffff04ff2cffff01ff648080ffff04ffff04ff12ffff04ff09ffff01ff8600e8d4a51000808080ffff04ffff04ff10ffff04ff15ffff04ff18ff80808080ff80808080ffff01ff04ffff04ff14ffff04ffff02ff2effff04ff02ffff04ff27ffff04ff57ffff04ff13ffff04ff2bff80808080808080ff808080ffff02ffff03ffff22ffff09ff2aff2f80ffff09ff15ff138080ffff01ff04ffff04ff12ffff04ff16ffff01ff8601d1a94a2001808080ff8080ff8080ff01808080ff0180ffff04ffff01ffffff329072657475726e20746f2073656e646572ff3dff5281ffffff33ff01ff8467616d658b7469632d7461632d746f65ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff3effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ff3affff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff3effff04ff02ffff04ff09ff80808080ffff02ff3effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01ffa03ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657bffb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a80ffff04ffff01ffb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a80ff01808080
opd ffffa0c8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35bafff8601d1a94a200180ff8080

(a (q 2 (q 2 (i (= 60 47) (q 4 (c 44 (q 100)) (c (c 18 (c 9 (q 0x00e8d4a51000))) (c (c 16 (c 21 (c 24 ()))) ()))) (q 4 (c 20 (c (a 46 (c 2 (c 39 (c 87 (c 19 (c 43 ())))))) ())) (a (i (all (= 42 47) (= 21 19)) (q 4 (c 18 (c 22 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 82 . -1) (51 1 "game" . "tic-tac-toe") 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 (a 62 (c 2 (c (c 5 (c 11 (c (c 58 (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) 2 (i (l 5) (q 11 (q . 2) (a 62 (c 2 (c 9 ()))) (a 62 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))
((0xc8e26acf4260

In [20]:
%%bash
brun '(a (q 2 (q 2 (i (= 60 47) (q 4 (c 44 (q 100)) (c (c 18 (c 9 (q 0x00e8d4a51000))) (c (c 16 (c 21 (c 24 ()))) ()))) (q 4 (c 20 (c (a 46 (c 2 (c 39 (c 87 (c 19 (c 43 ())))))) ())) (a (i (all (= 42 47) (= 21 19)) (q 4 (c 18 (c 22 (q 0x01d1a94a2001))) ()) ()) 1))) 1) (c (q ((50 . "return to sender") 61 82 . -1) (51 1 "game" . "tic-tac-toe") 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 (a 62 (c 2 (c (c 5 (c 11 (c (c 58 (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) 2 (i (l 5) (q 11 (q . 2) (a 62 (c 2 (c 9 ()))) (a 62 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) 1)))' '((0xc8e26acf4260c86b9373c0ae0f0f67f141413d58dee4f8b1346e2b21e8f35baf 0x01d1a94a2001) ())'

((61 0x43bc64fafddcee2b9788769a986277659d93dded0e908c18d9408fef201f17a8))


In [21]:
from chia.types.spend_bundle import SpendBundle
spend_bundle = SpendBundle([
        launcher_coin_spend,
        alice_waiting_room_coin_spend,
        bob_waiting_room_coin_spend
    ], 
    G2Element() # empty signature
)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_spends": [
        {
            "coin": {
                "amount": 2000000000001,
                "parent_coin_info": "0x18bc6ee8655344ffb0de25683cfee16b661aa28cfea66ae508327da877ec4a8e",
                "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
            },
            "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
            "solution": "0xffa0c8e26acf4260c86b9373c0ae0f0f67f141413d58d

## launcher_announcement:
`a061951257ac0f2b986a7dcd6f0cbb089321cd6991755a2bfad860b5bc49d1ee`

## launcher_coin_id:
`85032bd9190a0f4ad33a6a831c0e77519693bd0b8724b6cf5090949b7c7f4ab4`

## launcher_coin_announcement:
`149b602bd1cff4f9edf3813d24cc42dcdc83b0e3823b108e5579867f988729ec`

## cdv inspect
```
created  coin announcements = ['149b602bd1cff4f9edf3813d24cc42dcdc83b0e3823b108e5579867f988729ec']

asserted coin announcements = ['1be018c251f245fd24d0d843c1d65024256b8258222e7b64819b3d0e81f836b7', '1be018c251f245fd24d0d843c1d65024256b8258222e7b64819b3d0e81f836b7']

symdiff of coin announcements = ['149b602bd1cff4f9edf3813d24cc42dcdc83b0e3823b108e5579867f988729ec', '1be018c251f245fd24d0d843c1d65024256b8258222e7b64819b3d0e81f836b7']
```

### announcementID == sha256(coinID + message)

In [ ]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic-tac-toe"), 
            ("p1_pk", alice_pk),
            ("p2_pk", bob_pk)
        ]
    ]
)
print_program(launcher_solution)

launcher_announcement = launcher_solution.get_tree_hash()
print(launcher_announcement)